In [0]:
%pip install google-search-results
%pip install wikipedia

In [0]:
import os
from datetime import datetime

import openai
from langchain.llms import AzureOpenAI
from langchain.agents import initialize_agent, load_tools, ZeroShotAgent, Tool, AgentExecutor, load_tools
from langchain import OpenAI, SerpAPIWrapper, LLMChain

In [0]:
openai.api_type = "azure" 
openai.api_base =  "https://kf-llm-ins-2.openai.azure.com/" # Your Azure OpenAI resource's endpoint value.
openai.api_key = "49b50a14e4e647c39d4522d8c0774119"
os.environ["OPENAI_API_KEY"] = "49b50a14e4e647c39d4522d8c0774119"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_BASE"] = "https://kf-llm-ins-2.openai.azure.com/"
os.environ["SERPAPI_API_KEY"] = "4ea6865054c8eb8608723170dd33e12314d8e182c10c16ee15565ccf03526e8e"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "ls__c59a63b6ae494b5f97af20b35c96f711"
os.environ["LANGCHAIN_PROJECT"] = "tracing_simple_llm"

In [0]:
llm = AzureOpenAI(
    deployment_name="kf-gpt",
    model_name="gpt-35-turbo",temperature= 0.3
)

## Simple LLM Experiment to track in LangSmith

In [0]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: """

In [0]:
print(llm(prompt))

In [0]:
from langchain import PromptTemplate

template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

In [0]:
print(llm(
    prompt_template.format(
        query="Which libraries and model providers offer LLMs?"
    )
))

In [0]:
#Tools are functions that agents can use to interact with the world. These tools can be generic utilities (e.g. search), other chains, or even other agents.
tools = load_tools(["serpapi", "llm-math","wikipedia","terminal"], llm=llm) #The tools the agent will have access to are: [Google search, math, wikipedia, terminal]

In [0]:
agent = initialize_agent(tools,
                         llm,
                         agent="zero-shot-react-description",
                         verbose=True)
tools[1].name, tools[1].description

In [0]:
search = SerpAPIWrapper()
tools = [
   Tool(
       name="Search",
       func=search.run,
       description="useful for when you need to answer questions about current events",
   )
]

In [0]:
prefix = """Answer the following questions as best you can, but do it in old Shakepearean English. You have access to the following tools:"""
suffix = """Begin! Remember to speak in old Shakepearean English in the final answer. Use the word "behold" at least once.

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
   tools, prefix=prefix, suffix=suffix, input_variables=["input", "agent_scratchpad"]
)



print(prompt.template)

In [0]:
llm_chain = LLMChain(llm = AzureOpenAI(
    deployment_name="kf-gpt",
    model_name="gpt-35-turbo",temperature= 0.3
), prompt=prompt)

tool_names = [tool.name for tool in tools]

agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

agent_executor.run("How many hurricanes are expected to make landfall in the US this year?")

In [0]:
%pip install numexpr

In [0]:
# scenario 3 - Agent with Tools
# tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)
agent.run(
    "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
)